In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import numpy as np
import random
from bs4 import BeautifulSoup as bs
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import requests
import pandas as pd
import re
from selenium.common.exceptions import UnexpectedAlertPresentException
import urllib.request
import random
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import logging


In [ ]:
keywords = ['추천']

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Chrome 옵션 설정
options = Options()
options.add_argument('--disable-popup-blocking')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# WebDriver 실행 (ChromeDriver 버전이 Chrome 브라우저 버전과 일치하는지 확인)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


# 네이버 로그인 화면 이동
login_url = 'https://nid.naver.com/nidlogin.login'
driver.get(login_url)
driver.implicitly_wait(5)

# 아이디& 비밀번호 입력
my_id = 'choid0717' # "id" 대신에 자신의 네이버 아이디 입력
my_pw = 'Daun0717##' # "password" 대신에 자신의 네이버 비밀번호 입력

# 로그인 id, pw 입력
# 네이버에 로그인 할 경우 'send_keys()' 함수가 아니라 'execute_script()' 함수를 사용
driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
time.sleep(1)

# '로그인' 버튼 클릭
driver.find_element(By.XPATH, '//*[@id="log.login"]').click()
time.sleep(random.uniform(1,1.7))
url = 'https://cafe.naver.com/robotclear' # 크롤링할 카페 url 입력
driver.get(url)
time.sleep(1)

# iframe 진입
try:
    cafe_name = driver.find_element(By.CSS_SELECTOR, 'h2.cafe_name').text
    print("카페 이름:", cafe_name)
except Exception as e:
    print("카페 이름 추출 실패:", e)

In [ ]:
# 날짜 수집 함수
def extract_date(soup):
    try:
        date_element = soup.select_one('span.date')
        if date_element:
            date_text = date_element.text.strip()
            # 날짜 형식 처리
            if "시간 전" in date_text or "분 전" in date_text or "방금" in date_text:
                # 상대적 시간은 현재 날짜로 처리
                return time.strftime('%Y-%m-%d', time.localtime())
            elif "." in date_text:
                # YYYY.MM.DD 형식 -> YYYY-MM-DD로 변환
                return date_text.replace(".", "-")
        return "날짜 없음"  # 기본값
    except Exception as e:
        logging.error(f"날짜 추출 오류: {e}")
        return "날짜 없음"

# 본문 수집 함수
def extract_content(soup, include_images=True):
    contents = []

    # 다양한 본문 선택자 처리
    selectors = [
        'div.se-module.se-module-text',  # 일반 텍스트 본문
        'div.ContentRenderer',           # 특수 서식 본문
        'div.scrap_added',               # 스크랩 본문
    ]
    for selector in selectors:
        elements = soup.select(selector)
        if elements:
            contents.extend([element.text.strip() for element in elements])

    # 이미지 포함 여부에 따른 처리
    if include_images:
        images = soup.select('img')
        for img in images:
            if img.get('src'):
                contents.append(f"[이미지: {img['src']}]")  # 이미지 URL 포함

    return ' '.join(contents).strip() if contents else "본문 없음"

In [ ]:
# 제목, 본문, 댓글, 날짜, 검색 키워드, 게시판 이름 - 빈 list 생성
titles = []
reviews = []
comments = []
dates = []
search_keywords = []
board_titles = []
urls = []  # 게시글 URL 저장

## 제목만 추가 버전

In [ ]:
# 크롤링 시작
for keyword in keywords:
    print(f"\n📌 키워드 '{keyword}' 시작")

    for page in range(1, 41):
        url = f"https://cafe.naver.com/f-e/cafes/13411273/menus/25?viewType=L&ta=SUBJECT&q={keyword}&page={page}&size=50"
        driver.get(url)
        time.sleep(2)

        # 게시글 링크 수집
        try:
            post_links = [a.get_attribute('href') for a in driver.find_elements(By.CSS_SELECTOR, 'a.article')]
            if not post_links:
                print(f"📭 {page}페이지 게시글 없음, 중단")
                break
        except:
            continue

        for link in post_links:
            try:
                driver.get(link)
                time.sleep(random.uniform(1.2, 1.6))
                driver.switch_to.frame("cafe_main")
            except:
                continue

            try:
                soup = bs(driver.page_source, "lxml")

                title = soup.select_one("h3.title_text")
                board_title = soup.select_one("div.ArticleTitle")
                content = soup.select_one("div.se-main-container") or soup.select_one("div.ContentRenderer")
                date_elem = soup.select_one("span.date") or soup.select_one("span.se_publishDate")
                comment_elements = soup.find_all("span", class_="text_comment")

                titles.append(title.text.strip() if title else '')
                board_titles.append(board_title.text.strip() if board_title else '')
                reviews.append(content.get_text(strip=True) if content else '')
                dates.append(date_elem.text.strip() if date_elem else '')
                comments.append(" | ".join([c.text.strip() for c in comment_elements]) if comment_elements else '댓글 없음')
                search_keywords.append(keyword)
                urls.append(link)

                driver.switch_to.default_content()

            except:
                driver.switch_to.default_content()
                continue

In [ ]:
# 결과 저장
df = pd.DataFrame({
    'date': dates,
    'keyword': search_keywords,
    'title': titles,
    'contents': reviews,
    'comments': comments,
    'board_titles': board_titles,
    'url': urls,
})

df

In [ ]:
df.to_csv('naver_cafe_titleonly_crawling.csv', index=False, encoding='utf-8-sig')